# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
mongo.list_database_names()

['admin', 'config', 'epa', 'local', 'met', 'petsitly_marketing', 'uk_food']

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [6]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'St Marys Church Of England Primary School',
 'AddressLine2': 'Warren Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'St Marys COE (aided) Primary School',
 'BusinessType': 'School/college/university',
 'BusinessTypeID': 7845,
 'ChangesByServerID': 0,
 'Distance': 4591.871474174837,
 'FHRSID': 289353,
 'LocalAuthorityBusinessID': 'PI/000002468',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6QH',
 'RatingDate': '2016-09-13T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': {'$toInt': '$RatingValue'},
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65cd8bb823bd31358aba52c1'),
 'geocode': {'latitude': '51.085797', 'longitude': '1.194762'},
 'latitude': {'$toDouble': '$geocode.latitude'},
 'links': [{

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [9]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant_data)

InsertOneResult(ObjectId('65d40d1e4e1d464591c3b704'), acknowledged=True)

In [10]:
# Check that the new restaurant was inserted
query = {"BusinessName": {"$regex": "Penang Flavours"}}

# Capture the results to a variable
results = establishments.find(query)

# Print the results
for result in results:
    pprint(result)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RatingValue': {'$toInt': '$RatingValue'},
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65cee951c287b594e4ac5836'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'latitude': {'$toDouble': '$geocode.latitude'},
 'longitude': {'$toDouble': '$geocode.longitude'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'Addres

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
restaurant_type = db.establishments.find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, 
                                             {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1})
restaurant_type

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({"BusinessName": "Penang Flavours"}, 
                                          {"$set": {"BusinessTypeID": 1}})

UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [13]:
# Confirm that the new restaurant was updated
query = {"BusinessName": {"$regex": "Penang Flavours"}}

# Capture the results to a variable
results = establishments.find(query)

# Print the results
for result in results:
    pprint(result["BusinessTypeID"])

1
''
''
''
''
''
''
''


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": {"$regex": "Dover"}}

# Capture the results to a variable
count = establishments.count_documents(query)

# Print the results
print(count)

0


In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName": "Dover"})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [16]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": {"$regex": "Dover"}}

# Capture the results to a variable
count = establishments.count_documents(query)

# Print the results
print(count)

0


In [17]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()
pprint(remaining_document)

{'AddressLine1': 'St Marys Church Of England Primary School',
 'AddressLine2': 'Warren Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'St Marys COE (aided) Primary School',
 'BusinessType': 'School/college/university',
 'BusinessTypeID': 7845,
 'ChangesByServerID': 0,
 'Distance': 4591.871474174837,
 'FHRSID': 289353,
 'LocalAuthorityBusinessID': 'PI/000002468',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6QH',
 'RatingDate': '2016-09-13T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': {'$toInt': '$RatingValue'},
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65cd8bb823bd31358aba52c1'),
 'geocode': {'latitude': '51.085797', 'longitude': '1.194762'},
 'latitude': {'$toDouble': '$geocode.latitude'},
 'links': [{

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude and latitude
# This was provided by chatGPT
# Construct the update operation
update_operation = {
    '$set': {
        'latitude': {'$toDouble': '$geocode.latitude'},
        'longitude': {'$toDouble': '$geocode.longitude'}
    }
}

# Update multiple documents
result = establishments.update_many({}, update_operation)
result

UpdateResult({'n': 38797, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [19]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [20]:
# Change the data type from String to Integer for RatingValue
update_operation = {
    '$set': {
        'RatingValue': {'$toInt': '$RatingValue'}
    }
}

# Update multiple documents
result = establishments.update_many({}, update_operation)
result

UpdateResult({'n': 38797, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [21]:
# The documents appear to read " 'RatingValue': {'$toInt': '$RatingValue'}," in the RatingValue key. I'm not sure why and wasn't
# able to fix it by replacing the document or changing the code :(
# I tried converting the collection to a pandas DataFrame at one point to find the data type and the following was from chatGPT
# Check that the coordinates and rating value are now numbers
import pandas as pd

# Fetch all documents from the collection
cursor = establishments.find()

# Convert cursor to list of dictionaries
documents = list(cursor)

# Convert list of dictionaries to pandas DataFrame
df = pd.DataFrame(documents)

# Now you can work with the DataFrame 'df'
# Check the data type of the 'RatingValue' column
# Check the data type of 'RatingValue' column
print(df['RatingValue'].apply(lambda x: type(x)))

# Check the data type of 'latitude' column
print(df['geocode'].apply(lambda x: type(x['latitude']) if 'latitude' in x else None))

# Check the data type of 'longitude' column
print(df['geocode'].apply(lambda x: type(x['longitude']) if 'longitude' in x else None))



0        <class 'dict'>
1        <class 'dict'>
2        <class 'dict'>
3        <class 'dict'>
4        <class 'dict'>
              ...      
38792    <class 'dict'>
38793    <class 'dict'>
38794    <class 'dict'>
38795    <class 'dict'>
38796    <class 'dict'>
Name: RatingValue, Length: 38797, dtype: object


TypeError: argument of type 'float' is not iterable